In [15]:
import requests
import pandas as pd
import os
from tqdm import tqdm
from Bio import SeqIO
import numpy as np

In [2]:
df = pd.read_csv('../data/DeepPPI/SupplementaryS1.csv')

proteinA = list(df.proteinA.unique())
proteinB = list(df.proteinB.unique())
all_protein = list(set(proteinA+proteinB))
print('all proteins:', len(all_protein))

all proteins: 4424


In [3]:
f_name = '../data/DeepPPI/DeepPPI.fasta'
f = open(f_name, 'wb')
logger = []
for i, Id in tqdm(enumerate(all_protein), total=len(all_protein)):
    try:
        url = f'https://www.uniprot.org/uniprot/{Id}.fasta'
        record = requests.get(url, stream=True).content
        f.write(record)
    except Exception as e:
        logger.append((i,e))
        print(i, e)
f.close()

100%|██████████| 4424/4424 [50:56<00:00,  1.45it/s] 


In [10]:
fasta_path = '../data/DeepPPI/DeepPPI.fasta'

seq_dict = {}
for i, record in enumerate(tqdm(SeqIO.parse(fasta_path, 'fasta'), total=4424)):
    Id = record.id.split('|')[1]
    seq = record.seq
    seq_dict[str(Id)] = seq

100%|██████████| 4424/4424 [00:00<00:00, 27112.89it/s]


In [33]:
proteinA_seq = []
proteinB_seq = []
for IdA, IdB in zip(df['proteinA'], df['proteinB']):
    proteinA_seq.append(str(seq_dict[IdA]))
    proteinB_seq.append(str(seq_dict[IdB]))

In [36]:
df['proteinA_seq'] = proteinA_seq
df['proteinB_seq'] = proteinB_seq

In [38]:
df.to_csv('../data/DeepPPI/DeepPPIAll.csv', index=False)